# Easy quantities with IfcOpenShell Selectors

Do the basic import of necessary modules

In [ ]:
!git clone https://github.com/jakob-beetz/sbe-2025-lca-workshop.git

In [ ]:
!pip install ifcopenshell
%cd sbe-2025-lca-workshop

In [ ]:
import ifcopenshell
import pandas as pd

Let's import the Duplex Model
If you would like to work on Hilo model instead, uncomment the second line

In [ ]:
m = ifcopenshell.open("/content/sbe-2025-lca-workshop/data/IFC_models/Duplex_QTO.ifc")
# m = ifcopenshell.open("/content/sbe-2025-lca-workshop/data/IFC_models/HiLo.ifc")

In [ ]:
from IPython.display import IFrame
IFrame(src='https://app.speckle.systems/projects/8d7bac3e0a/models/69621ea0d8', width=1200, height=600)

Let's inspect the Model. Get all the spaces

In [ ]:
m.by_type("IfcSpace")

Ok, these are the raw entity instances as they appear in the IFC text (SPFF) file. Not very readable for the non-technical reader.

Let's just list the names:

In [ ]:
for space in m.by_type("IfcSpace"):
    print(space.LongName)

In [ ]:
for wall in m.by_type("IfcWall"):
    print(wall.Name)

# Procedural way  to walk throught the model
For each wall, list all possible ways to define materials in IFC

In [ ]:
for wall in m.by_type("IfcWall"):
  for rel in wall.HasAssociations:
    if rel.is_a("IfcRelAssociatesMaterial"):
      if rel.RelatingMaterial.is_a("IfcMaterial"):
        print(f"Wall: {wall.Name}, Material: {rel.RelatingMaterial.Name}")
      elif rel.RelatingMaterial.is_a("IfcMaterialList"):
        material_names = [mat.Name for mat in rel.RelatingMaterial.Materials]
        print(f"Wall: {wall.Name}, Materials: {', '.join(material_names)}")
      elif rel.RelatingMaterial.is_a("IfcMaterialLayerSetUsage"):
        material_names = [layer.Material.Name for layer in rel.RelatingMaterial.ForLayerSet.MaterialLayers]
        print(f"Wall: {wall.Name}, Material Layers: {', '.join(material_names)}")
      elif rel.RelatingMaterial.is_a("IfcMaterialConstituentSet"):
          constituents = []
          for constituent in rel.RelatingMaterial.MaterialConstituents:
              if constituent.Material:
                  constituents.append(constituent.Material.Name)
          print(f"Wall: {wall.Name}, Material Constituents: {', '.join(constituents)}")

# Selector
Selectors of ifcopenshell are a way to query the model in a more human readable way.

In [ ]:
import ifcopenshell.util.selector
wall = m.by_type("IfcWall")[3]
# Get the Name attribute of the wall's type.
print(ifcopenshell.util.selector.get_element_value(wall, "material.Name"))
print(ifcopenshell.util.selector.get_element_value(wall, "Dimensions.Area"))

# Export to tables and pandas dataframes

see the `all_materials_query_duplex.json` file in the `IFC_Models` directory and adjust to your needs
Code taken from Dion Moults IfcCSV contributions for Bonsai and wrapped for easy execution in a notebook

In [ ]:
!pip install ifccsv
import ifccsv
from utils import IfcCsvWrap as ifccsv

df = ifccsv.get_ifc_csv("/content/sbe-2025-lca-workshop/data/IFC_models/Duplex_QTO.ifc", "/content/sbe-2025-lca-workshop/data/IFC_models/all_materials_query_duplex.json")
df

In [ ]:
# Convert 'Area' column to numeric, coercing errors
df['Area'] = pd.to_numeric(df['Area'], errors='coerce')


# Group by 'mat.0' and sum the 'Area'
grouped_df = df.groupby('mat.1')['Area'].sum().reset_index()

# Rename the summed column for clarity (optional)
grouped_df = grouped_df.rename(columns={'Area': 'Total Area'})

# Print or display the result
print("Grouped and summarized Area by mat.0:")
grouped_df